In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit
from scipy.integrate import cumtrapz

In [ ]:
def clean_data(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()

    values = []
    for line in lines:
        try:
            value = float(line.strip())
            if value <= 100: # Values above 100 degrees Celsius are caused by an error of the device and are ommited. 
                values.append(value)
        except ValueError:
            pass
    
    # If a value is higher by 0.5 degrees Celsius than the average of the adjacent values, it is deleted. 
    filtered_values = []
    for i in range(1, len(values)-1):
        if (values[i-1] + values[i+1]) / 2 >= values[i] - 0.5:
            filtered_values.append(values[i])

    x_all = [0.4 * k for k in range(len(filtered_values))]

    plt.plot(x_all, filtered_values, marker='o', label='Temperature')
    plt.xlabel('Time (s)')
    plt.ylabel('Temperature (°C)')
    plt.title('Temperature over Time with Regression Line')
    plt.legend()
    plt.grid(True)
    plt.show()

    return filtered_values, x_all

In [ ]:
def find_max(filtered_values, x_all):
    max_index = np.argmax(filtered_values)
    max_x_value = x_all[max_index]
    max_y_value = filtered_values[max_index]

    return max_x_value, max_y_value, max_index

In [ ]:
def exponential_decay(x, a, b, c):
    return a * np.exp(-(1 / b) * x) + c

In [ ]:
def perform_linear_fitting(y_values, required_R2, points_for_fitting):
    for i in range(len(y_values) - points_for_fitting):
        x_values_fit = [0.4 * j for j in range(i, i + points_for_fitting)]
        y_values_fit = y_values[i:i + points_for_fitting]

        # Perform linear regression. 
        model = LinearRegression()
        x_values_reshaped = np.array(x_values_fit).reshape(-1, 1)
        model.fit(x_values_reshaped, y_values_fit)

        # Calculate adjusted R-squared. 
        predictions = model.predict(x_values_reshaped)
        r2_adj = 1 - (1 - r2_score(y_values_fit, predictions)) * (
            (len(y_values_fit) - 1) / (len(y_values_fit) - len(model.coef_) - 1)
        )

        # If adjusted R-squared is greater than or equal to a certain value, plot the graph and return. 
        if r2_adj >= required_R2:
            x_all_fit = [0.4 * k for k in range(len(y_values))]
            regression_line = model.predict(np.array(x_all_fit).reshape(-1, 1))

            # Save intercept and slope
            intercept = model.intercept_
            slope = model.coef_[0]

            plt.plot(x_all_fit, y_values, marker='o', label='Temperature')
            plt.plot(x_all_fit[:points_for_fitting], regression_line[:points_for_fitting], color='red', label='Regression Line')

            plt.xlabel('Time (s)')
            plt.ylabel('Temperature (°C)')
            plt.title('Temperature over Time with Regression Line')
            plt.legend()
            plt.grid(True)
            plt.show()

            return x_all_fit, y_values, intercept, slope, r2_adj, regression_line
 
    return None

In [ ]:
# First run calculations for the distilled water. 

file_name = input("Enter the file name of water measurements: ") 
#file_name = '.txt'  

filtered_values, x_all_w = clean_data(file_name)

max_x_value_w, max_y_value_w, max_index_w = find_max(filtered_values, x_all_w)
print("X value where temperature is maximum:", max_x_value_w)
print("Maximum temperature:", max_y_value_w)
print("Index where max value occures:", max_index_w)

x_fit = np.array(x_all_w[max_index_w:])
y_fit = np.array(filtered_values[max_index_w:])
popt_w, _ = curve_fit(exponential_decay, x_fit, y_fit, p0=[50, 300, 23], maxfev=10000)

# Extract the fitted parameters. 
a, b, c = popt_w

# Generate points for the fitted curve. 
fit_curve = exponential_decay(x_fit, *popt_w)

# Print the optimized parameters. 
print(f"Fitted parameters: a = {a}, b = {b}, c = {c}")

# Plot the original data up to the maximum point and the fitted curve. 
plt.plot(x_all_w, filtered_values, marker='o', label='Original Data')
plt.plot(x_all_w[max_index_w:], fit_curve, label='Exponential Decay Fit', color='red')
plt.xlabel('Time (s)')
plt.ylabel('Temperature (°C)')
plt.title('Exponential Decay Fit')
plt.legend()
plt.grid(True)
plt.show()

# Calculate the new y-values based on the formula. 
new_y_values = (np.array(filtered_values) - c) / b 

# Calculate the first-order derivative. 
first_order_derivative = np.gradient(filtered_values, x_all_w) 

# Add new_y_values and first_order_derivative element-wise. 
data_prior_integration = new_y_values + first_order_derivative 

# Cumulative trapezoidal integration using scipy. 
data_after_integration_water = cumtrapz(data_prior_integration, x=x_all_w, initial=0) 

In [ ]:
# Now compute the required data from the sample's measurement. 

file_name = input("Enter the file name: ") 
#file_name = '.txt' 

concentration = float(input("Enter the nanoparticle concentration values: ")) 
#concentration = 1.0 # mg / mL units 

#required_R2 = 0.99 
required_R2 = input("Enter a required adjusted R-squared value: ") 

# If the fitting is incorrect, lower the required_R2, the points_for_fitting or both. 

#points_for_fitting = 400 
points_for_fitting = input("Enter the number of points of the linear fitting: ") 

In [ ]:
filtered_values, x_all = clean_data(file_name) 

max_x_value, max_y_value, max_index = find_max(filtered_values, x_all) 
print("X value where temperature is maximum:", max_x_value) 
print("Maximum temperature:", max_y_value) 
print("Index where max value occures:", max_index) 

In [ ]:
x_fit = np.array(x_all[max_index:]) 
y_fit = np.array(filtered_values[max_index:]) 
popt, _ = curve_fit(exponential_decay, x_fit, y_fit, p0=[50, 300, 23], maxfev=10000) 

# Extract the fitted parameters. 
a, b, c = popt 

# Generate points for the fitted curve. 
fit_curve = exponential_decay(x_fit, *popt) 

In [ ]:
# Print the optimized parameters. 
print(f"Fitted parameters: a = {a}, b = {b}, c = {c}") 

# Plot the original data up to the maximum point and the fitted curve. 
plt.plot(x_all, filtered_values, marker='o', label='Original Data') 
plt.plot(x_all[max_index:], fit_curve, label='Exponential Decay Fit', color='red') 
plt.xlabel('Time (s)') 
plt.ylabel('Temperature (°C)') 
plt.title('Exponential Decay Fit') 
plt.legend() 
plt.grid(True) 
plt.show() 

In [ ]:
# Calculate the new y-values based on the formula. 
new_y_values = (np.array(filtered_values) - c) / b 

# Calculate the first-order derivative. 
first_order_derivative = np.gradient(filtered_values, x_all) 

# Add new_y_values and first_order_derivative element-wise. 
data_prior_integration = new_y_values + first_order_derivative 

# Cumulative trapezoidal integration using scipy. 
data_after_integration = cumtrapz(data_prior_integration, x=x_all, initial=0) 

In [ ]:
# Check if everything is going as planned with the respective plots. 

# Create subplots. 
fig, axs = plt.subplots(1, 2, figsize=(12, 5)) 

# Plot first_order_derivative. 
axs[0].plot(x_all, first_order_derivative, label='First Order Derivative', color='green') 
axs[0].set_xlabel('Time (s)') 
axs[0].set_ylabel('First Order Derivative') 
axs[0].set_title('First Order Derivative') 
axs[0].legend() 
axs[0].grid(True) 

# Plot data_after_integration. 
axs[1].plot(x_all, data_after_integration, label='Integrated Values', linestyle='-.', color='blue') 
axs[1].set_xlabel('Time (s)') 
axs[1].set_ylabel('Integrated Values') 
axs[1].set_title('Integrated Values After Combining new_y_values and First-Order Derivative') 
axs[1].legend() 
axs[1].grid(True) 

# Adjust layout to prevent clipping of titles. 
plt.tight_layout() 
plt.show() 

In [ ]:
# Trim y-axis and x-axis data to acquire matching lengths. 
if len(data_after_integration) > len(data_after_integration_water):
    # Trim data_after_integration to match the length of data_after_integration_water. 
    data_after_integration_trimmed = data_after_integration[-len(data_after_integration_water):]
    x_all_trimmed = x_all[-len(x_all_w):]
    adiabatic_curve = data_after_integration_trimmed - data_after_integration_water
    
elif len(data_after_integration) < len(data_after_integration_water):
    # Trim data_after_integration_water to match the length of data_after_integration
    data_after_integration_trimmed_water = data_after_integration_water[-len(data_after_integration):]
    x_all_trimmed_w = x_all_w[-len(x_all):]
    adiabatic_curve = data_after_integration_trimmed - data_after_integration_trimmed_water
    
else: 
    adiabatic_curve = data_after_integration - data_after_integration_water

In [ ]:
# Compute and plot the adiabatic curve of the sample. 

# Plot adiabatic_curve. 
plt.plot(x_all_trimmed, adiabatic_curve, label='Adiabatic Curve', linestyle='-', color='red')
plt.xlabel('Time (s)')
plt.ylabel('Adiabatic Curve')
plt.title('Adiabatic Curve')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Obtain the linear fitting results. 
result = perform_linear_fitting(adiabatic_curve, required_R2, points_for_fitting) 

temp = True 

if result: 
    x_all, filtered_values, intercept, slope, r2_adj, regression_line = result 
    print("Linear Regression Results:") 
    print("Intercept:", intercept) 
    print("Slope:", slope) 
    print("Adjusted R-squared:", r2_adj) 
else: 
    print(f"No regression with adjusted R-squared >= {required_R2} found.") 
    temp = False 

In [ ]:
# Specific heat of water. 
C_H2O = 4.186 # J / (g * K) units. 

# Calculating the Specific Loss Power (SLP) index. 
if temp:
    SLP = (C_H2O * slope) / (concentration / 1000) # Watt per gram units. 
    print("The SLP of this sample is: ", SLP, "W/g") 